In [19]:
%pip install transformers
%pip install datasets
%pip install torch torchvision torchaudio
%pip install ipywidgets



Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


<h2>Data Processing</h2>

In [20]:
import pandas as pd
import os
directory = '/training_datasets_dir/'

<h2>Retrain RoBERTa model</h2>

In [21]:
# files is a list of files in the training_dir that we want to process
def processfiles(files, training_dir):
    texts = []
    for i, file in enumerate(files):
        text = ""
        try:
            with open(os.path.join(training_dir, file)) as instream:
                for line in instream:
                    text += line
                texts.append(text)
        except UnicodeDecodeError:
            print(f"Unicode error for file {file}")
    return texts

In [22]:
import glob
# the /* does all the files in the directory (I'm pretty sure...)
file_names = glob.glob('./training_datasets_dir/*.txt')
file_names

['./training_datasets_dir\\sample.txt', './training_datasets_dir\\sample2.txt']

In [23]:
# glob is outputting it with the entire path name already, so the directory in this case is just blank since we are running it from the src folder already
texts = processfiles(file_names, "")

<h2>Creating hugging face dataset from text files</h2>

In [25]:
from transformers import AutoTokenizer
import datasets
from datasets import Dataset

OSError: [WinError 126] The specified module could not be found. Error loading "C:\Users\derek\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\lib\shm.dll" or one of its dependencies.

In [ ]:
texts_dict = {'text': [t for t in texts]}

In [ ]:
dataset = Dataset.from_dict(texts_dict)

In [ ]:
class TokenizerWrapper:
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer
    
    def tokenize_function(self, examples):
        return self.tokenizer(
            examples["text"]
        )

In [ ]:
model_name = 'distilroberta-base' # for more accuracy use the base roberta model not the distilled version
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer_wrapper = TokenizerWrapper(tokenizer)
# tokenized_dataset = dataset.map(lambda batch: tokenizer(batch['text']), batched=True, num_proc=2, remove_columns=['text'])
tokenized_dataset = dataset.map(tokenizer_wrapper.tokenize_function, batched=True, num_proc=2, remove_columns=['text'])

Map (num_proc=2): 100%|██████████| 2/2 [00:05<00:00,  2.83s/ examples]


In [ ]:
def group_texts(b):
    block_size = 128
    concat_text = {k: sum(b[k], []) for k in b.keys()}
    total_length = len(concat_text[list(b.keys())[0]])
    total_length = (total_length // block_size) * block_size # drops any remainder we have when we divide by block_size
    result = {k: [t[i: i + block_size] for i in range(0, total_length, block_size)] for k, t in concat_text.items()}
    result['labels'] = result['input_ids'].copy()
    return result

In [ ]:
blocked_dataset = tokenized_dataset.map(group_texts, batched=True, num_proc=2, batch_size=1000)

Map (num_proc=2): 100%|██████████| 2/2 [00:03<00:00,  1.51s/ examples]


In [27]:
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15) #0.15 is the default value

OSError: [WinError 126] The specified module could not be found. Error loading "C:\Users\derek\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\lib\shm.dll" or one of its dependencies.

In [26]:
data_collator([blocked_dataset.__getitem__(0)])

NameError: name 'data_collator' is not defined